In [14]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

# Load pre-trained model and tokenizer
model_name = "t5-small"  # You can replace this with "gpt2" or any other model you prefer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

# Fine-tune parameters
learning_rate = 1e-4
batch_size = 4
epochs = 3

# Define optimizer and loss function (only defined once)
optimizer = tf.keras.optimizers.Adam(learning_rate)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [7]:


# Prepare your data (texts and summaries)
train_texts = [
    "The new iPhone 14 is set to launch next month with revolutionary features. Apple enthusiasts are eagerly anticipating its release. The new features promise to redefine the smartphone experience. Rumors suggest it will have advanced camera capabilities. Analysts predict record-breaking sales for the highly anticipated device.",
    "Scientists have discovered a potential cure for Alzheimer's disease using gene therapy. The breakthrough offers hope to millions of patients and their families. Clinical trials have shown promising results in reversing cognitive decline. Researchers are optimistic about the long-term effectiveness of the treatment. Further studies are underway to refine the therapy and bring it to market.",
    "The stock market experienced a major dip today due to concerns over inflation. Investors are worried about rising prices and their impact on the economy. Analysts attribute the downturn to uncertainty surrounding monetary policy. Market volatility is expected to continue in the coming days. Traders are closely monitoring key economic indicators for signs of recovery.",
    "A new study suggests that regular exercise can significantly reduce the risk of heart disease. Physical activity has long been associated with cardiovascular health. The findings highlight the importance of staying active for overall well-being. Researchers recommend incorporating exercise into daily routines for optimal benefits. Public health initiatives are promoting fitness to combat heart disease.",
    "The United Nations has called for urgent action to address the climate crisis. Global leaders are urged to implement bold measures to curb greenhouse gas emissions. Climate change poses a threat to ecosystems and human civilization. Environmental activists are demanding immediate steps to reduce carbon footprints. Sustainable development goals aim to mitigate the impact of climate change.",
    "The latest Marvel movie broke box office records, earning over $1 billion in its opening weekend. Fans flocked to theaters to experience the highly anticipated film. The movie received critical acclaim for its action-packed scenes and visual effects. Box office success solidifies the franchise's position as a cultural phenomenon. Sequels are already in development to continue the epic saga.",
    "Researchers have developed a breakthrough technology for clean energy production. The innovation promises to revolutionize the renewable energy sector. Sustainable solutions are crucial for combating climate change. The new technology harnesses natural resources to generate electricity. Governments and corporations are investing in clean energy initiatives to reduce carbon emissions.",
    "A famous chef has revealed the secret recipe to his award-winning dish. Food enthusiasts can now recreate the culinary masterpiece at home. The recipe features a blend of exotic spices and fresh ingredients. Cooking tutorials offer step-by-step instructions for novice chefs. The dish is praised for its rich flavors and gourmet presentation.",
    "SpaceX successfully launched its latest rocket, bringing supplies to the International Space Station. The mission marks another milestone for private space exploration. The rocket's payload includes scientific experiments and essential provisions for astronauts. SpaceX continues to lead the commercial space industry with its innovative technology. Future missions aim to expand human presence in space.",
    "A new art exhibit featuring the works of renowned artists opened to the public today. Art enthusiasts gathered to admire the diverse collection of paintings and sculptures. The exhibit showcases the creativity and talent of contemporary artists. Curators provide insights into the artistic process and inspirations behind the works. Visitors are encouraged to engage with the artwork and explore different perspectives.",
    "The government announced plans to invest billions in infrastructure projects to boost the economy. The initiative aims to create jobs and stimulate economic growth. Infrastructure improvements will enhance transportation networks and public utilities. Funding allocations prioritize projects with long-term benefits for communities. Government officials emphasize the importance of infrastructure investments in driving prosperity.",
    "A team of scientists has discovered a new species of dinosaur fossil in Antarctica. The find offers valuable insights into prehistoric ecosystems and evolution. Paleontologists are studying the fossilized remains to classify the new species. The discovery sheds light on the diversity of dinosaurs during the Mesozoic era. Expeditions continue to uncover fossils in remote regions of the world.",
    "The World Health Organization declared a global health emergency in response to a new viral outbreak. Health officials are racing to contain the spread of the virus. International cooperation is essential for coordinating response efforts and sharing resources. Public awareness campaigns educate communities about preventive measures and symptoms. Healthcare systems are mobilizing to provide medical care and support to affected individuals.",
    "The latest fashion trends from Paris Fashion Week have captivated audiences worldwide. Designers showcased innovative designs and bold statements on the runway. Fashionistas eagerly await the release of new collections from top brands. Influencers and celebrities set trends with their glamorous ensembles and accessories. Fashion Week celebrates creativity and individuality in the world of haute couture.",
    "A leading tech company unveiled its newest product, promising to revolutionize the industry. The innovative device boasts cutting-edge features and sleek design. Tech enthusiasts eagerly anticipate its release date and pre-order availability. Early reviews praise the product's performance and user-friendly interface. The company aims to disrupt the market with its state-of-the-art technology.",
    "A popular streaming service announced a lineup of original content for the upcoming year. Subscribers can look forward to a diverse selection of movies and TV shows. The streaming platform invests in high-quality productions to attract viewers. Original series and films cater to a wide range of interests and demographics. Content creators collaborate with top talent to deliver compelling entertainment experiences.",
    "The CEO of a major corporation stepped down amidst allegations of misconduct. The resignation comes amid a corporate scandal and internal investigation. Shareholders express concerns over the company's reputation and governance practices. Interim leadership is appointed to oversee operations and restore trust. Legal proceedings and regulatory inquiries are underway to address the allegations.",
    "A groundbreaking study has shed light on the origins of the universe. Astrophysicists unravel the mysteries of cosmic evolution and cosmic microwave background radiation. The findings provide insights into the Big Bang theory and cosmic inflation. Scientific models offer explanations for the formation of galaxies and cosmic structures. The study advances our understanding of the fundamental laws of physics.",
    "A renowned author released their highly anticipated novel, receiving rave reviews from critics. The book captivates readers with its compelling storyline and vivid characters. Literary accolades recognize the author's mastery of storytelling and narrative technique. Book signings and author events draw crowds of eager fans and book enthusiasts. The novel becomes a bestseller and cultural phenomenon.",
    "The automotive industry is embracing electric vehicles as the future of transportation. Automakers invest in electric vehicle technology to meet consumer demand and regulatory standards. Electric cars offer environmental benefits and cost savings on fuel consumption. Charging infrastructure expands to support widespread adoption of electric vehicles. Governments incentivize electric vehicle purchases to accelerate the transition to sustainable transportation.",
    "A famous landmark underwent extensive renovations to preserve its cultural heritage. The restoration project aims to protect the historical significance of the iconic site. Conservation efforts focus on preserving architectural features and artwork for future generations. Tourists and visitors are invited to witness the transformation of the landmark. The reopening ceremony celebrates the completion of the restoration work.",
    "The winner of this year's Nobel Prize in Literature was announced, honoring a prolific writer. The prestigious award recognizes the author's contributions to literature and storytelling. The recipient is celebrated for their unique voice and literary achievements. Nobel laureates join the ranks of literary legends in the pantheon of great writers. The award ceremony pays tribute to the power of words and imagination.",
    "A humanitarian organization launched a campaign to provide aid to refugees in war-torn regions. The initiative aims to alleviate suffering and address urgent humanitarian needs. Donations support food, shelter, and medical assistance for displaced families and individuals. Volunteers and aid workers distribute relief supplies to refugee camps and communities. The campaign raises awareness about the plight of refugees and calls for global solidarity.",
    "A new smartphone app aims to revolutionize the way people manage their finances. The app offers users a comprehensive suite of tools for budgeting and money management. Features include expense tracking, bill payment reminders, and investment portfolio management. User-friendly interface and intuitive design make financial management accessible to everyone. The app empowers users to take control of their finances and achieve financial goals.",
    "The latest advancements in artificial intelligence are reshaping industries worldwide. AI technologies enable automation, data analysis, and predictive modeling across various sectors. Businesses leverage AI to improve efficiency, productivity, and decision-making processes. AI-powered applications transform customer service, marketing, and logistics operations. Technological innovation drives economic growth and competitiveness in the global marketplace.",
    "An acclaimed film director is set to release their latest project, generating buzz among cinephiles. The upcoming movie features an ensemble cast and gripping storyline. Trailers and teasers generate anticipation and speculation among moviegoers. Film festivals and premieres showcase the director's creative vision and cinematic expertise. Critical acclaim and box office success propel the movie to awards season contention.",
    "A groundbreaking medical procedure offers hope to patients with previously untreatable conditions. The innovative treatment utilizes gene therapy and regenerative medicine techniques. Clinical trials demonstrate promising results in restoring health and function to patients. Medical breakthroughs pave the way for personalized therapies and precision medicine. Healthcare providers collaborate to expand access to cutting-edge treatments and improve patient outcomes.",
    "The government passed a landmark bill to protect endangered species and their habitats. Conservation measures aim to preserve biodiversity and ecosystems for future generations. Legislators collaborate with environmental groups and stakeholders to draft comprehensive wildlife protection laws. Public awareness campaigns raise awareness about endangered species and conservation efforts. Conservationists celebrate the passage of legislation as a milestone for environmental conservation.",
    "A leading environmentalist warned of the urgent need to address deforestation in the Amazon rainforest. The destruction of vital ecosystems threatens biodiversity and indigenous communities. Advocates call for sustainable land management practices and forest conservation initiatives. International agreements and partnerships mobilize resources for rainforest protection and restoration. Environmental activists raise awareness about the consequences of deforestation on global climate.",
    "The world's largest technology conference kicked off with keynote speeches from industry leaders. Tech enthusiasts and professionals gather to explore the latest innovations and trends. Conference sessions cover topics such as AI, cybersecurity, and blockchain technology. Exhibitors showcase cutting-edge products and solutions in the expo hall. Networking events and panel discussions facilitate collaboration and knowledge sharing among attendees.",
    "A team of researchers made a major breakthrough in the fight against cancer. The discovery offers new insights into tumor biology and cancer treatment strategies. Experimental therapies show promising results in targeting cancer cells and preventing metastasis. Clinical trials are underway to evaluate the effectiveness and safety of novel treatments. Collaborative efforts accelerate progress in cancer research and bring hope to patients.",
    "A new museum exhibition showcases the rich cultural history of indigenous peoples. Artifacts, artworks, and interactive displays highlight the diverse traditions and heritage of indigenous communities. Curators collaborate with tribal leaders and historians to provide accurate and respectful representation. Educational programs and workshops engage visitors in learning about indigenous cultures and issues. The exhibition promotes cultural appreciation and understanding among visitors.",
    "The latest smartphone model boasts cutting-edge features and sleek design. Tech enthusiasts eagerly anticipate the release of the highly anticipated device. Rumors and leaks provide insights into the device's specifications and capabilities. Early reviews praise the smartphone's performance and user experience. The smartphone sets new standards for innovation and functionality in the mobile industry.",
    "An international summit convened to address the growing threat of cyber warfare. Global leaders discuss strategies for cybersecurity and cyber defense in the digital age. Diplomatic efforts focus on establishing norms and rules for responsible behavior in cyberspace. Collaborative initiatives aim to enhance information sharing and coordination among nations. The summit reaffirms commitments to protecting critical infrastructure and preserving peace and stability online.",
    "A renowned chef shared their favorite recipes for healthy and delicious meals. Cooking enthusiasts can now recreate gourmet dishes at home with ease. The recipes feature fresh, seasonal ingredients and simple cooking techniques. Cooking tutorials provide step-by-step instructions and cooking tips from the chef. Home cooks are inspired to explore new flavors and culinary techniques in the kitchen.",
    "The fashion industry is embracing sustainability with eco-friendly clothing lines. Fashion brands prioritize ethical sourcing and environmentally friendly production methods. Sustainable fashion promotes transparency and accountability in the supply chain. Consumers demand eco-conscious fashion choices to reduce environmental impact. Fashion designers innovate with recycled materials and low-impact dyeing processes to create stylish and sustainable clothing.",
    "A landmark court ruling set a precedent for protecting civil liberties in the digital age. The decision affirms privacy rights and limits government surveillance powers. Legal experts analyze the implications of the ruling on data privacy and freedom of expression. Civil rights advocates applaud the court's decision as a victory for individual rights and democracy. The ruling establishes legal safeguards to protect online privacy and personal data.",
    "A team of scientists discovered a potential treatment for antibiotic-resistant bacteria. The breakthrough offers hope for combating the growing threat of drug-resistant infections. Experimental drugs show efficacy in killing antibiotic-resistant strains of bacteria. Clinical trials are underway to evaluate the safety and effectiveness of the new treatments. Medical researchers collaborate to develop novel antibiotics and alternative therapies to combat antibiotic resistance.",
    "The tourism industry is rebounding with a surge in travel bookings after pandemic restrictions ease. Travelers are eager to explore new destinations and embark on adventures. Tourism operators and hospitality businesses adapt to changing traveler preferences and safety protocols. Destinations roll out promotional campaigns to attract visitors and boost local economies. Tourism contributes to recovery efforts and revitalizes communities affected by the pandemic.",
    "An influential artist unveiled a series of thought-provoking installations exploring social issues. The art installations challenge perceptions and provoke critical reflection on contemporary issues. Art critics praise the artist's creativity and boldness in addressing societal issues. The artist collaborates with community groups and activists to raise awareness and spark dialogue. Art exhibitions draw diverse audiences and inspire social change through the power of art.",
    "The CEO of a tech startup announced plans for expansion into new markets. The company seeks to capitalize on emerging opportunities and diversify revenue streams. Expansion strategies include partnerships, acquisitions, and product innovation. Investors show confidence in the company's growth prospects and market potential. The CEO outlines a vision for the company's future and commitment to delivering value to shareholders.",
    "A groundbreaking study revealed the impact of climate change on global food security. Scientists warn of disruptions to food production and distribution systems due to extreme weather events. Vulnerable populations face increased risk of hunger and malnutrition as crops fail and prices rise. Policy makers are urged to implement adaptive measures and invest in sustainable agriculture practices. Global cooperation is essential to mitigate the impact of climate change on food systems.",
    "A new documentary sheds light on the plight of endangered species around the world. Filmmakers document conservation efforts and wildlife habitats in remote regions. The documentary raises awareness about the threats facing endangered species and ecosystems. Viewers are inspired to take action to protect biodiversity and preserve natural habitats. Environmental organizations collaborate to promote conservation initiatives and wildlife protection.",
    "The latest breakthroughs in renewable energy offer hope for a sustainable future. Innovations in solar, wind, and hydroelectric power revolutionize the energy landscape. Renewable energy technologies reduce reliance on fossil fuels and mitigate climate change. Governments and corporations invest in clean energy infrastructure and research to accelerate the transition to renewable sources. Sustainable energy solutions pave the way for a greener and more resilient planet.",
    "A celebrity philanthropist pledged millions to fund education initiatives in underserved communities. The donation supports scholarships, mentorship programs, and educational resources for students in need. Philanthropic efforts aim to close the opportunity gap and empower disadvantaged youth through education. Schools and nonprofits collaborate to implement innovative learning initiatives and support student success. The philanthropist's generosity inspires others to invest in education as a pathway to social mobility.",
    "The fashion industry is embracing body positivity with diverse representation on the runway. Fashion brands celebrate diversity and inclusivity in their advertising campaigns and fashion shows. Models of all shapes, sizes, and backgrounds showcase the latest trends and styles. Body-positive movements challenge beauty standards and promote self-acceptance and empowerment. Fashion designers create clothing that celebrates individuality and promotes confidence and self-expression.",
    "A team of engineers developed a new technology for harvesting clean energy from ocean waves. The innovative device converts wave energy into electricity using renewable power generation techniques. Ocean energy holds vast potential as a sustainable energy source for coastal communities. Pilot projects demonstrate the feasibility and scalability of wave energy technologies. Governments and energy companies invest in ocean energy research and development to unlock its full potential.",
    "The winner of this year's Pulitzer Prize for Fiction was announced, recognizing a talented author. The prestigious literary award honors excellence in storytelling and narrative craft. The winning novel captivates readers with its compelling characters and thought-provoking themes. The author's literary achievements are celebrated by fellow writers and readers alike. The Pulitzer Prize ceremony showcases the best in American literature and fosters appreciation for the written word.",
    "A renowned musician released their highly anticipated album to critical acclaim. The album showcases the artist's musical evolution and creative vision. Fans praise the album for its lyrical depth and sonic innovation. Music critics hail the artist's talent and contribution to contemporary music. The album's release sparks excitement and anticipation for upcoming concerts and performances.",
    "The gaming industry is booming with the release of next-generation consoles and blockbuster titles. Gamers eagerly await the latest releases and updates from their favorite developers. Next-gen consoles offer immersive gaming experiences with advanced graphics and gameplay features. Blockbuster titles break sales records and dominate online gaming platforms. Esports tournaments attract millions of viewers and showcase the competitive spirit of gaming.",
   
]

train_summaries = [
    "The new iPhone 14, with revolutionary features, is set to launch next month, anticipated eagerly by Apple enthusiasts.",
    "Scientists discover a potential cure for Alzheimer's disease using gene therapy, showing promising results in reversing cognitive decline.",
    "Concerns over inflation cause a major dip in the stock market today, attributed to uncertainty surrounding monetary policy.",
    "Regular exercise significantly reduces the risk of heart disease, highlighting the importance of physical activity for overall well-being.",
    "The United Nations urges urgent action to address the climate crisis, emphasizing the need for global cooperation and sustainable development goals.",
    "The latest Marvel movie breaks box office records, receiving critical acclaim and solidifying its position as a cultural phenomenon.",
    "Breakthrough technology for clean energy production promises to revolutionize the renewable energy sector, crucial for combating climate change.",
    "A famous chef reveals the secret recipe to his award-winning dish, allowing food enthusiasts to recreate the culinary masterpiece at home.",
    "SpaceX successfully launches its latest rocket, marking another milestone in private space exploration and advancing future missions.",
    "A new art exhibit featuring renowned artists opens to the public, showcasing diverse artworks and engaging visitors in cultural exploration.",
    "Government announces plans to invest billions in infrastructure projects to stimulate economic growth and create jobs.",
    "Discovery of a new species of dinosaur fossil in Antarctica offers valuable insights into prehistoric ecosystems and evolution.",
    "World Health Organization declares a global health emergency in response to a new viral outbreak, emphasizing the need for international cooperation.",
    "Latest fashion trends from Paris Fashion Week captivate global audiences, celebrating creativity and individuality in haute couture.",
    "Leading tech company unveils its newest product, aiming to revolutionize the industry with cutting-edge features and innovation.",
    "Popular streaming service announces lineup of original content for the upcoming year, offering diverse entertainment options for subscribers.",
    "CEO of a major corporation steps down amidst allegations of misconduct, prompting concerns over governance practices and shareholder trust.",
    "Groundbreaking study sheds light on the origins of the universe, advancing scientific understanding of cosmic evolution.",
    "Renowned author releases highly anticipated novel, receiving critical acclaim and becoming a cultural phenomenon.",
    "Automotive industry embraces electric vehicles as the future of transportation, driving innovation and sustainability efforts.",
    "Famous landmark undergoes extensive renovations to preserve its cultural heritage, attracting tourists and celebrating history.",
    "Winner of this year's Nobel Prize in Literature is announced, honoring a talented author's contributions to storytelling.",
    "Humanitarian organization launches campaign to provide aid to refugees, addressing urgent humanitarian needs and raising global awareness.",
    "New smartphone app aims to revolutionize financial management, offering comprehensive tools for budgeting and money management.",
    "Advancements in artificial intelligence reshape industries worldwide, driving efficiency and innovation across sectors.",
    "Acclaimed film director's latest project generates buzz among cinephiles, showcasing creative vision and cinematic expertise.",
    "Groundbreaking medical procedure offers hope to patients with untreatable conditions, advancing personalized medicine.",
    "Landmark bill passed to protect endangered species and habitats, marking a milestone for environmental conservation.",
    "Leading environmentalist warns of urgent need to address deforestation in the Amazon rainforest, advocating for sustainable land management.",
    "Largest technology conference kicks off with keynote speeches, facilitating collaboration and showcasing innovation.",
    "Breakthrough in cancer research offers new treatment strategies, providing hope for patients and advancing scientific knowledge.",
    "Museum exhibition highlights cultural history of indigenous peoples, promoting cultural appreciation and understanding.",
    "Latest smartphone model boasts cutting-edge features, setting new standards for innovation and functionality.",
    "International summit convenes to address threat of cyber warfare, emphasizing the importance of cybersecurity and global cooperation.",
    "Renowned chef shares favorite recipes, inspiring home cooks to explore new flavors and culinary techniques.",
    "Fashion industry embraces sustainability with eco-friendly clothing lines, promoting ethical sourcing and environmentally friendly production methods.",
    "Landmark court ruling sets precedent for protecting civil liberties in the digital age, affirming privacy rights and freedom of expression.",
    "Discovery of potential treatment for antibiotic-resistant bacteria offers hope for combating drug-resistant infections.",
    "Tourism industry rebounds with surge in travel bookings, contributing to economic recovery and revitalization.",
    "Influential artist unveils thought-provoking installations, sparking critical reflection and dialogue on social issues.",
    "CEO of tech startup announces plans for expansion, driving growth and innovation in new markets.",
    "Study reveals impact of climate change on global food security, emphasizing need for adaptive measures and sustainable agriculture.",
    "Documentary sheds light on plight of endangered species, raising awareness and promoting conservation efforts.",
    "Breakthroughs in renewable energy offer hope for a sustainable future, reducing reliance on fossil fuels.",
    "Celebrity philanthropist pledges millions to fund education initiatives, empowering disadvantaged youth through education.",
    "Fashion industry embraces body positivity, celebrating diversity and promoting self-acceptance.",
    "Engineers develop technology for harvesting clean energy from ocean waves, advancing sustainable energy solutions.",
    "Winner of this year's Pulitzer Prize for Fiction recognized for literary excellence, celebrating the power of storytelling.",
    "Renowned musician releases highly anticipated album, showcasing musical talent and innovation.",
    "Gaming industry booms with release of next-generation consoles and blockbuster titles, driving innovation and entertainment."
]


In [17]:
# Tokenize and pad the data (consider max_length for padding)
max_length = 512  # Adjust based on your data

# Option 1: Loop through both lists together (recommended for clarity)
tokenized_train = []
for text_list, summary in zip(train_texts, train_summaries[:, :-1]):
  # If text_list is a list:
  text = " ".join(text_list)  # Join elements into a single string
  encoded_pair = tokenizer(text, summary, padding=True, truncation=True, return_tensors="tf")
  tokenized_train.append(encoded_pair)

input_ids = [x["input_ids"] for x in tokenized_train]
attention_mask = [x["attention_mask"] for x in tokenized_train]
decoder_attention_mask = tf.math.not_equal(input_ids, tokenizer.pad_token_id)  # Ignore padding tokens

# Option 2: Concatenate and then tokenize (alternative approach)
# combined_text = train_texts + train_summaries[:, :-1]
# tokenized_train = tokenizer(combined_text, padding=True, truncation=True, return_tensors="tf")
# input_ids = tokenized_train["input_ids"]
# attention_mask = tokenized_train["attention_mask"]
# decoder_attention_mask = tf.math.not_equal(input_ids, tokenizer.pad_token_id)  # Ignore padding tokens

# Fine-tune the model
for epoch in range(epochs):
  total_loss = 0
  for i in range(0, len(train_texts), batch_size):
    with tf.GradientTape() as tape:
      outputs = model(input_ids[i:i + batch_size], attention_mask=attention_mask[i:i + batch_size], decoder_attention_mask=decoder_attention_mask[i:i + batch_size])
      logits = outputs.logits
      shifted_labels = input_ids[i:i + batch_size, 1:]
      loss = loss_fn(shifted_labels, logits[:, :-1, :])  # Ignore padding in logits

    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    total_loss += loss.numpy()

  avg_loss = total_loss / (len(train_texts) // batch_size)
  print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")

TypeError: list indices must be integers or slices, not tuple